# DeepLabCut Toolbox
https://github.com/AlexEMG/DeepLabCut

Nath\*, Mathis\* et al. *Using DeepLabCut for markerless pose estimation during behavior across species*, (under revision).

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.
This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to:
- create a project
- extract training frames
- label the frames
- plot the labeled images
- create a training set
- train a network
- evaluate a network
- analyze a novel video
- create an automatically labeled video 
- plot the trajectories

*Note*: Refine a network based after the network was trained on just a few labeled images is illustrated in "Demo-labeledexample-MouseReaching.ipynb". This demo also contains an already labeled data set and is perhaps the best starting point for brand new users.

## Create a new project

It is always good idea to keep the projects separate. This function creates a new project with subdirectories and a basic configuration file in the user defined directory otherwise the project is created in the current working directory.

You can always add new videos to the project at any stage of the project. 

In [28]:
import datetime
import deeplabcut

In [28]:
task='frontslowmo' # Enter the name of your experiment Task
experimenter='vj' # Enter the name of the experimenter
video=[r"C:\Users\vjj14\Downloads\1080p.MOV"] # Enter the paths of your videos you want to grab frames from.
time_log= open("time_log{}.txt".format(task),"w+")
path_config_file=deeplabcut.create_new_project(task,experimenter,video, working_directory= r'C:\Users\vjj14\Desktop\DeepLabCut',copy_videos=True) #change the working directory to where you want the folders created.

# The function returns the path, where your project is. 
# You could also enter this manually (e.g. if the project is already created and you want to pick up, where you stopped...)
#path_config_file = '/home/Mackenzie/Reaching/config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)

Created "C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-06\videos"
Created "C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-06\labeled-data"
Created "C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-06\training-datasets"
Created "C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-06\dlc-models"
Copying the videos
C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-06\videos\1080p.MOV
Generated "C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-06\config.yaml"

A new project with name frontslowmo-vj-2019-06-06 is created at C:\Users\vjj14\Desktop\DeepLabCut and a configurable file (config.yaml) is stored there. Change the parameters in this file to adapt to your project's needs.
 Once you have changed the configuration file, use the function 'extract_frames' to select frames for labeling.
. [OPTIONAL] Use the function 'add_new_videos' to add new videos to your project (at any stage).


In [31]:
path_config_file='C:\\Users\\vjj14\\Desktop\\DeepLabCut\\frontslowmo-vj-2019-06-05\\config.yaml'
def write_log(text, new_line=True):
    time_log= open("time_log{}.txt".format('frontslowmo'),"a+")
    if new_line:
        time_log.write("\n")
    time_log.write(text)
    time_log.close()

## Extract frames from videos 
A key point for a successful feature detector is to select diverse frames, which are typical for the behavior you study that should be labeled.

This function selects N frames either uniformly sampled from a particular video (or folder) (algo=='uniform'). Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.

Also make sure to get select data from different (behavioral) sessions and different animals if those vary substantially (to train an invariant feature detector).

Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later as well, it is advisable to crop the frames, to remove unnecessary parts of the frame as much as possible.

Always check the output of cropping. If you are happy with the results proceed to labeling.

In [34]:
start = datetime.datetime.now()
write_log(str(start) + "***extract_frames start")

%matplotlib inline
deeplabcut.extract_frames(path_config_file,'automatic','uniform',crop=False) #there are other ways to grab frames, such as by clustering 'kmeans'; please see the paper. 

end = datetime.datetime.now()
write_log(str(end) + "***extract_frames end")
write_log("   time elapsed:" + str(end-start))

Config file read successfully.
Do you want to extract (perhaps additional) frames for video: C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\videos\1080p.MOV ?
yes/nono

Frames were selected.
You can now label the frames using the function 'label_frames' (if you extracted enough frames for all videos).


In [21]:
time_log.close()
time_log= open("time_log{}.txt".format('frontslowmo'),"a+")

## Label the extracted frames
Only videos in the config file can be used to extract the frames. Extracted labels for each video are stored in the project directory under the subdirectory **'labeled-data'**. Each subdirectory is named after the name of the video. The toolbox has a labeling toolbox which could be used for labeling. 

In [4]:
start = datetime.datetime.now()
write_log(str(start) + "***label_frames start")

%gui wx
deeplabcut.label_frames(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***label_frames end")
write_log("   time elapsed:" + str(end-start))

ERROR:root:Cannot activate multiple GUI eventloops


ImportError: Cannot load backend 'TkAgg' which requires the 'tk' interactive framework, as 'wx' is currently running

**Check the labels**

Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. The DeepLabCut toolbox provides a function `check\_labels'  to do so. It is used as follows:

In [10]:
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by vj.
C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\labeled-data\1080p_labeled  already exists!
They are stored in the following folder: C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\labeled-data\1080p_labeled.
Attention: C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\labeled-data\1080p does not appear to have labeled data!
If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


If the labels need adjusted, you can use the refinement GUI to move them around! Check that out below.

## Create a training dataset
This function generates the training data information for DeepCut (which requires a mat file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [12]:
start = datetime.datetime.now()
write_log(str(start) + "***create_training_dataset start")

deeplabcut.create_training_dataset(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***create_training_dataset end")
write_log("   time elapsed:" + str(end-start))

C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\training-datasets\iteration-0\UnaugmentedDataSet_frontslowmoJun5  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


In [6]:
?deeplabcut.create_training_dataset

## Start training - If you want to use a CPU, continue. 
### If you want to use your GPU, you need to exit here and either work from the Docker container, your own TensorFlow installation in an Anaconda env

This function trains the network for a specific shuffle of the training dataset. 

In [35]:
start = datetime.datetime.now()
write_log(str(start) + "***train_network start")
time_log.write("   DESCRIPTION:" + input("Training description(Network, Iterations, etc) "))

deeplabcut.train_network(path_config_file, saveiters=1000, displayiters=100, maxiters=300000)

end = datetime.datetime.now()
write_log(str(end) + "***train_network end")
write_log("   time elapsed:" + str(end-start))

Training description(Iteration, Iterations, etc) n


KeyboardInterrupt: 

In [43]:
#iter0  ~ 1 hour and 20 minutes
#iter1 start-1:07pm

In [18]:
?deeplabcut.train_network

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [8]:
start = datetime.datetime.now()
write_log(str(start) + "***evaluate_network start")

deeplabcut.evaluate_network(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***evalaute_network end")
write_log("   time elapsed:" + str(end-start))

C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05/evaluation-results/  already exists!


Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['LeftHand', 'RightHand', 'Nose', 'Pellet'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-2\\UnaugmentedDataSet_frontslowmoJun5\\frontslowmo_vj95shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\vjj14\\.conda\\envs\\dlc-windowsGPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets\\iteration-2\\UnaugmentedDataSet_frontslowmoJun5\\Documentation_data-frontslowmo_95shuffle1.pickl

Running  DeepCut_resnet50_frontslowmoJun5shuffle1_26000  with # of trainingiterations: 26000
INFO:tensorflow:Restoring parameters from C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\dlc-models\iteration-2\frontslowmoJun5-trainset95shuffle1\train\snapshot-26000


INFO:tensorflow:Restoring parameters from C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\dlc-models\iteration-2\frontslowmoJun5-trainset95shuffle1\train\snapshot-26000


Analyzing data...


253it [00:44,  5.48it/s]


Done and results stored for snapshot:  snapshot-26000
Results for 26000  training iterations: 95 1 train error: 12.5 pixels. Test error: 32.18  pixels.
With pcutoff of 0.1  train error: 12.5 pixels. Test error: 32.18 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)


## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [15]:
videofile_path = [r"C:\Users\vjj14\Downloads\compressed25.mp4"] #Enter the list of videos to analyze.

start = datetime.datetime.now()
write_log(str(start) + "***analyze_videos start")
write_lof("   VIDEOS: " + str(videofile_path))

deeplabcut.analyze_videos(path_config_file,videofile_path)

end = datetime.datetime.now()
write_log(str(end) + "***analyze_videos end")
write_log("   time elapsed:" + str(end-start))

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['LeftHand', 'RightHand', 'Nose', 'Pellet'],
 'batch_size': 4,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-1\\UnaugmentedDataSet_frontslowmoJun5\\frontslowmo_vj95shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\vjj14\\.conda\\envs\\dlc-windowsGPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets\\iteration-2\\UnaugmentedDataSet_frontslowmoJun5\\Documentation_data-frontslowmo_95shuffle1.pickl

Using snapshot-28000 for model C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\dlc-models\iteration-1\frontslowmoJun5-trainset95shuffle1
INFO:tensorflow:Restoring parameters from C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\dlc-models\iteration-1\frontslowmoJun5-trainset95shuffle1\train\snapshot-28000


INFO:tensorflow:Restoring parameters from C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\dlc-models\iteration-1\frontslowmoJun5-trainset95shuffle1\train\snapshot-28000


Starting to analyze %  C:\Users\vjj14\Downloads\compressed25.mp4
Loading  C:\Users\vjj14\Downloads\compressed25.mp4
Duration of video [s]:  518.7 , recorded with  30.0 fps!
Overall # of frames:  15561  found with (before cropping) frame dimensions:  1920 1080
Starting to extract posture




  0%|                                                                                        | 0/15561 [00:00<?, ?it/s]

  2%|█▌                                                                            | 310/15561 [00:12<10:12, 24.90it/s]

  2%|█▌                                                                            | 310/15561 [00:24<10:12, 24.90it/s]

  3%|██▎                                                                           | 465/15561 [00:25<13:12, 19.05it/s]

  4%|███                                                                           | 620/15561 [00:37<15:16, 16.31it/s]

  5%|███▉                                                                          | 775/15561 [00:50<16:36, 14.83it/s]

  6%|████▋                                                                         | 930/15561 [01:02<17:15, 14.13it/s]

  6%|████▋                                                                         | 930/15561 [01:14<17:15, 14.13it/s]

  7%|█████▎                   

Detected frames:  15561
Saving results in C:\Users\vjj14\Downloads...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!


In [ ]:
#Keep Track of time for analyzing videos
#Original: 940 mb
#25:134 mb, took 20:55

In [13]:
?deeplabcut.analyze_videos

 16%|████████████▎                                                                | 2480/15561 [03:30<17:57, 12.14it/s]

## Extract outlier frames [optional step]
This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. Make sure to provide the correct value of the "iterations" as it will be used to create the unique directory where the extracted frames will be saved.

In [8]:
alg = 'uncertain'
param = 0.0002 #epsilon or p_bound

start = datetime.datetime.now()
write_log(str(start) + "***extract_outlier_frames start")

deeplabcut.extract_outlier_frames(path_config_file, videofile_path, outlieralgorithm=alg, p_bound=param)
num_frames = input("number of frames selected?")
write_log("   VIDEOS: {0}, ALGORITHM: {1}, PARAMETER: {2}, NUM FRAMES: {3}".format(str(videofile_path), alg, param, num_frames))

end = datetime.datetime.now()
write_log(str(end) + "***analyze_videos end")
write_log("   time elapsed:" + str(end-start))

network parameters: DeepCut_resnet50_frontslowmoJun5shuffle1_28000
Method  uncertain  found  12  putative outlier frames.
Do you want to proceed with extracting  200  of those?
If this list is very large, perhaps consider changing the paramters (start, stop, p_bound, comparisonbodyparts) or use a different method.
yes/noyes
Frames from video 1080p  already extracted (more will be added)!
Loading video...
Duration of video [s]:  518.6666666666666 , recorded @  30.0 fps!
Overall # of frames:  15560 with (cropped) frame dimensions: 
Kmeans-quantization based extracting of frames from 0.0  seconds to 518.7  seconds.
Let's select frames indices: [113, 154, 155, 156, 157, 158, 160, 161, 162, 13116, 13117, 13126]
New video was added to the project! Use the function 'extract_frames' to select frames for labeling.
The outlier frames are extracted. They are stored in the subdirectory labeled-data\1080p.
Once you extracted frames for all videos, use 'refine_labels' to manually correct the labels.

In [3]:
videofile_path = [r"C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\videos\1080p.MOV"]
videofile_path

['C:\\Users\\vjj14\\Desktop\\DeepLabCut\\frontslowmo-vj-2019-06-05\\videos\\1080p.MOV']

In [38]:
?deeplabcut.extract_outlier_frames

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset.

In [4]:
start = datetime.datetime.now()
write_log(str(start) + "***refine_frames start")

%gui wx
deeplabcut.refine_labels(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***refine_frames end")
write_log("   time elapsed:" + str(end-start))


ERROR:root:Cannot activate multiple GUI eventloops


Windows
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Closing... The refined labels are stored in a subdirectory under labeled-data. Use the function 'merge_datasets' to augment the training dataset, and then re-train a network using create_training_dataset followed by train_network!


In [5]:
#Once all folders are relabeled, check them and advance. See how to check labels, above!
start = datetime.datetime.now()
write_log(str(start) + "***create_training_dataset start")

deeplabcut.create_training_dataset(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***create_training_dataset end")
write_log("   time elapsed:" + str(end-start))

Merged data sets and updated refinement iteration to 2.
Now you can create a new training set for the expanded annotated images (use create_training_dataset).


## Create a new iteration of training dataset [optional step]
Following the refine labels, append these frames to the original dataset to create a new iteration of training dataset.

In [6]:
start = datetime.datetime.now()
write_log(str(start) + "***refine_frames start")

deeplabcut.create_training_dataset(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***refine_frames end")
write_log("   time elapsed:" + str(end-start))

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [16]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

Starting %  C:\Users\vjj14\Downloads ['C:\\Users\\vjj14\\Downloads\\compressed25.mp4']
Loading  C:\Users\vjj14\Downloads\compressed25.mp4 and data.
False 0 1920 0 1080
15561
Duration of video [s]:  518.7 , recorded with  30.0 fps!
Overall # of frames:  15561 with cropped frame dimensions:  1920 1080
Generating frames and creating video.




  0%|                                                                                        | 0/15561 [00:00<?, ?it/s]

  0%|                                                                                | 4/15561 [00:00<06:36, 39.20it/s]

  0%|                                                                               | 11/15561 [00:00<05:46, 44.94it/s]

  0%|                                                                               | 17/15561 [00:00<05:20, 48.49it/s]

  0%|                                                                               | 24/15561 [00:00<05:00, 51.76it/s]

  0%|▏                                                                              | 31/15561 [00:00<04:40, 55.42it/s]

  0%|▏                                                                              | 37/15561 [00:00<04:41, 55.25it/s]

  0%|▏                                                                              | 44/15561 [00:00<04:27, 57.93it/s]

  0%|▎                        

  6%|█████                                                                         | 998/15561 [00:14<03:23, 71.48it/s]

  6%|████▉                                                                        | 1006/15561 [00:15<03:21, 72.28it/s]

  7%|█████                                                                        | 1014/15561 [00:15<03:21, 72.07it/s]

  7%|█████                                                                        | 1022/15561 [00:15<03:21, 72.06it/s]

  7%|█████                                                                        | 1030/15561 [00:15<03:16, 74.00it/s]

  7%|█████▏                                                                       | 1038/15561 [00:15<03:18, 73.22it/s]

  7%|█████▏                                                                       | 1046/15561 [00:15<03:18, 73.17it/s]

  7%|█████▏                                                                       | 1054/15561 [00:15<03:19, 72.67it/s]

  7%|█████▎                     

 13%|██████████▏                                                                  | 2068/15561 [00:29<03:06, 72.32it/s]

 13%|██████████▎                                                                  | 2076/15561 [00:29<03:06, 72.48it/s]

 13%|██████████▎                                                                  | 2084/15561 [00:30<03:06, 72.20it/s]

 13%|██████████▎                                                                  | 2092/15561 [00:30<03:06, 72.25it/s]

 13%|██████████▍                                                                  | 2100/15561 [00:30<03:07, 71.94it/s]

 14%|██████████▍                                                                  | 2108/15561 [00:30<03:06, 71.98it/s]

 14%|██████████▍                                                                  | 2116/15561 [00:30<03:06, 72.14it/s]

 14%|██████████▌                                                                  | 2124/15561 [00:30<03:02, 73.53it/s]

 14%|██████████▌                

 20%|███████████████▍                                                             | 3118/15561 [00:44<02:55, 71.06it/s]

 20%|███████████████▍                                                             | 3126/15561 [00:44<02:50, 72.96it/s]

 20%|███████████████▌                                                             | 3134/15561 [00:44<02:51, 72.46it/s]

 20%|███████████████▌                                                             | 3142/15561 [00:44<02:51, 72.21it/s]

 20%|███████████████▌                                                             | 3150/15561 [00:45<02:52, 72.02it/s]

 20%|███████████████▋                                                             | 3158/15561 [00:45<02:52, 72.02it/s]

 20%|███████████████▋                                                             | 3166/15561 [00:45<02:52, 72.00it/s]

 20%|███████████████▋                                                             | 3174/15561 [00:45<02:52, 71.72it/s]

 20%|███████████████▋           

...

 73%|███████████████████████████████████████████████████████▏                    | 11309/15561 [02:43<01:01, 69.33it/s]

 73%|███████████████████████████████████████████████████████▎                    | 11317/15561 [02:43<01:00, 69.63it/s]

 73%|███████████████████████████████████████████████████████▎                    | 11325/15561 [02:43<01:00, 70.40it/s]

 73%|███████████████████████████████████████████████████████▎                    | 11333/15561 [02:43<01:00, 70.02it/s]

 73%|███████████████████████████████████████████████████████▍                    | 11341/15561 [02:43<01:00, 69.76it/s]

 73%|███████████████████████████████████████████████████████▍                    | 11348/15561 [02:43<01:03, 66.70it/s]

 73%|███████████████████████████████████████████████████████▍                    | 11355/15561 [02:43<01:06, 62.97it/s]

 73%|███████████████████████████████████████████████████████▍                    | 11363/15561 [02:43<01:04, 65.18it/s]

 73%|███████████████████████████

 79%|████████████████████████████████████████████████████████████▏               | 12329/15561 [02:58<00:45, 71.09it/s]

 79%|████████████████████████████████████████████████████████████▎               | 12337/15561 [02:58<00:45, 71.25it/s]

 79%|████████████████████████████████████████████████████████████▎               | 12345/15561 [02:58<00:45, 71.36it/s]

 79%|████████████████████████████████████████████████████████████▎               | 12353/15561 [02:58<00:45, 70.87it/s]

 79%|████████████████████████████████████████████████████████████▎               | 12361/15561 [02:58<00:45, 70.90it/s]

 79%|████████████████████████████████████████████████████████████▍               | 12369/15561 [02:58<00:44, 71.12it/s]

 80%|████████████████████████████████████████████████████████████▍               | 12377/15561 [02:58<00:45, 70.52it/s]

 80%|████████████████████████████████████████████████████████████▍               | 12385/15561 [02:58<00:45, 70.47it/s]

 80%|███████████████████████████

 86%|█████████████████████████████████████████████████████████████████           | 13329/15561 [03:12<00:34, 64.11it/s]

 86%|█████████████████████████████████████████████████████████████████▏          | 13336/15561 [03:12<00:34, 64.90it/s]

 86%|█████████████████████████████████████████████████████████████████▏          | 13343/15561 [03:13<00:33, 65.84it/s]

 86%|█████████████████████████████████████████████████████████████████▏          | 13350/15561 [03:13<00:33, 65.77it/s]

 86%|█████████████████████████████████████████████████████████████████▏          | 13357/15561 [03:13<00:32, 66.84it/s]

 86%|█████████████████████████████████████████████████████████████████▎          | 13365/15561 [03:13<00:32, 67.86it/s]

 86%|█████████████████████████████████████████████████████████████████▎          | 13372/15561 [03:13<00:32, 68.14it/s]

 86%|█████████████████████████████████████████████████████████████████▎          | 13380/15561 [03:13<00:31, 68.79it/s]

 86%|███████████████████████████

 92%|██████████████████████████████████████████████████████████████████████      | 14351/15561 [03:27<00:16, 71.99it/s]

 92%|██████████████████████████████████████████████████████████████████████▏     | 14359/15561 [03:27<00:16, 72.46it/s]

 92%|██████████████████████████████████████████████████████████████████████▏     | 14367/15561 [03:27<00:16, 72.40it/s]

 92%|██████████████████████████████████████████████████████████████████████▏     | 14375/15561 [03:27<00:16, 72.76it/s]

 92%|██████████████████████████████████████████████████████████████████████▏     | 14383/15561 [03:28<00:16, 72.22it/s]

 92%|██████████████████████████████████████████████████████████████████████▎     | 14391/15561 [03:28<00:16, 72.43it/s]

 93%|██████████████████████████████████████████████████████████████████████▎     | 14399/15561 [03:28<00:15, 72.77it/s]

 93%|██████████████████████████████████████████████████████████████████████▎     | 14407/15561 [03:28<00:15, 72.82it/s]

 93%|███████████████████████████

 99%|███████████████████████████████████████████████████████████████████████████▏| 15387/15561 [03:42<00:02, 60.35it/s]

 99%|███████████████████████████████████████████████████████████████████████████▏| 15394/15561 [03:42<00:02, 61.67it/s]

 99%|███████████████████████████████████████████████████████████████████████████▏| 15401/15561 [03:42<00:02, 63.31it/s]

 99%|███████████████████████████████████████████████████████████████████████████▎| 15408/15561 [03:42<00:02, 64.15it/s]

 99%|███████████████████████████████████████████████████████████████████████████▎| 15415/15561 [03:42<00:02, 64.76it/s]

 99%|███████████████████████████████████████████████████████████████████████████▎| 15422/15561 [03:43<00:02, 64.29it/s]

 99%|███████████████████████████████████████████████████████████████████████████▎| 15429/15561 [03:43<00:02, 63.28it/s]

 99%|███████████████████████████████████████████████████████████████████████████▍| 15436/15561 [03:43<00:01, 63.26it/s]

 99%|███████████████████████████

In [22]:
?deeplabcut.create_labeled_video

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [45]:
videofile_path

['C:\\Users\\vjj14\\Desktop\\DeepLabCut\\frontslowmo-vj-2019-06-05\\videos\\1080p.MOV']

In [ ]:
%matplotlib notebook 
#for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)

In [58]:
?deeplabcut.plot_trajectories

In [68]:
path_config_file

'C:\\Users\\vjj14\\Desktop\\DeepLabCut\\frontslowmo-vj-2019-06-05\\config.yaml'

In [72]:
import numpy as np
import pandas as pd

In [74]:
df = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\videos\1080pDeepCut_resnet50_frontslowmoJun5shuffle1_30000.h5")
df.describe()

scorer    DeepCut_resnet50_frontslowmoJun5shuffle1_30000                \
bodyparts                                       LeftHand                 
coords                                                 x             y   
count                                       15560.000000  15560.000000   
mean                                          971.243753    725.853752   
std                                           171.896374    139.390499   
min                                            -3.870381      0.415488   
25%                                           914.894261    649.265245   
50%                                          1024.541650    783.569860   
75%                                          1064.598903    821.210082   
max                                          1611.307688   1088.421771   

scorer                                                             \
bodyparts                   RightHand                               
coords       likelihood             x             y    likelihood   
count      15560.000000  15560.000000  15560.000000  15560.000000   
mean           0.699717    906.089441    738.766505      0.503567   
std            0.357561    156.673818    166.830793      0.400728   
min            0.000027    319.477097      1.081914      0.000040   
25%            0.397388    850.580156    651.611459      0.060324   
50%            0.910440    905.468754    778.514918      0.466857   
75%            0.986921    940.134475    858.174915      0.949224   
max            0.999329   1609.412701   1092.391703      0.999041   

scorer                                                             \
bodyparts          Nose                                    Pellet   
coords                x             y    likelihood             x   
count      15560.000000  15560.000000  15560.000000  15560.000000   
mean         931.766309    656.488581      0.671245    957.103689   
std          169.306891    191.976523      0.363396    159.386299   
min          298.748627     -0.059892      0.000104     -5.040360   
25%          909.805872    595.394240      0.306699    922.235798   
50%          918.396317    703.317179      0.875269    939.903313   
75%          928.343641    794.728728      0.960582    953.739810   
max         1623.930874   1088.734671      0.998151   1613.409633   

scorer                                 
bodyparts                              
coords                y    likelihood  
count      15560.000000  15560.000000  
mean         811.306070      0.322458  
std          177.523694      0.418997  
min            0.012989      0.000098  
25%          671.503496      0.027885  
50%          847.006353      0.048265  
75%          940.497040      0.954270  
max         1089.468144      0.997180

In [15]:
df = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\frontslowmo-vj-2019-06-05\videos\1080pDeepCut_resnet50_frontslowmoJun5shuffle1_30000.h5")
df.describe()

scorer    DeepCut_resnet50_front4k60fpszoom3.0_TrimJun3shuffle1_286000  \
bodyparts                                                     LeftHand   
coords                                                               x   
count                                             612.000000             
mean                                             1294.587569             
std                                               566.489027             
min                                                -0.173059             
25%                                               955.338525             
50%                                              1458.549845             
75%                                              1644.293059             
max                                              2161.044888             

scorer                                                                    \
bodyparts                             RightHand                            
coords               y  likelihood            x            y  likelihood   
count       612.000000  612.000000   612.000000   612.000000  612.000000   
mean       2455.822662    0.042246   609.627308  2811.817150    0.078364   
std         890.589556    0.138305   523.969844   763.555947    0.202453   
min           1.988913    0.000032     0.665848    76.401595    0.000153   
25%        1989.080632    0.000861     3.883312  2168.396138    0.001087   
50%        2089.116435    0.002397   764.576838  2395.076544    0.003136   
75%        3421.296761    0.011128   970.543810  3556.132724    0.019450   
max        3840.645847    0.977641  2139.572770  3840.410774    0.992057   

scorer                                                                     \
bodyparts         Nose                                Pellet                
coords               x            y  likelihood            x            y   
count       612.000000   612.000000  612.000000   612.000000   612.000000   
mean       1000.140640  2138.825481    0.222379  1161.420468  3607.459208   
std         249.147238   615.631056    0.313997   635.214989   435.197877   
min           0.269251   865.347540    0.000097     2.184450  1940.036492   
25%         990.724893  1820.097459    0.004228   736.731850  3637.395378   
50%        1030.763698  1986.446780    0.050586   963.540944  3822.212681   
75%        1069.846140  2166.098999    0.300523  1822.975545  3837.997508   
max        2152.967607  3841.093842    0.998524  2158.995823  3841.865475   

scorer                 
bodyparts              
coords     likelihood  
count      612.000000  
mean         0.364173  
std          0.308807  
min          0.000428  
25%          0.070970  
50%          0.276980  
75%          0.658898  
max          0.989263

In [71]:
df = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\front4k60fpszoom3.0_Trim-nv-2019-06-03\videos\front4k60fpszoom3.0_Trim2DeepCut_resnet50_front4k60fpszoom3.0_TrimJun3shuffle1_286000.h5")
df.describe()

scorer    DeepCut_resnet50_front4k60fpszoom3.0_TrimJun3shuffle1_286000  \
bodyparts                                                     LeftHand   
coords                                                               x   
count                                             396.000000             
mean                                             1081.024990             
std                                               801.865156             
min                                                -1.075624             
25%                                               306.234073             
50%                                              1099.686393             
75%                                              1978.196430             
max                                              2162.322889             

scorer                                                                    \
bodyparts                             RightHand                            
coords               y  likelihood            x            y  likelihood   
count       396.000000  396.000000   396.000000   396.000000  396.000000   
mean       1915.892243    0.028620   507.260805  2028.133410    0.074291   
std         925.669503    0.089883   570.154718   789.584933    0.167471   
min           4.104054    0.000054     1.491787     0.110741    0.000132   
25%        1363.246884    0.001314     6.579469  1460.181150    0.001657   
50%        1541.223487    0.004285   137.076892  1945.901693    0.008166   
75%        2062.913943    0.013648   941.707332  2366.580038    0.052594   
max        3840.759733    0.904895  2135.639879  3840.026480    0.958066   

scorer                                                                     \
bodyparts         Nose                                Pellet                
coords               x            y  likelihood            x            y   
count       396.000000   396.000000  396.000000   396.000000   396.000000   
mean        832.015528  2042.639285    0.122190  1359.938273  3524.089973   
std         477.821564   905.199187    0.237828   693.038635   567.594545   
min          -1.233883     1.529313    0.000029     1.956876  1154.500621   
25%         795.323176  1530.546586    0.000845   649.109436  3461.410198   
50%         980.203261  1763.745589    0.006032  1651.501211  3830.206299   
75%        1059.898767  2137.143606    0.097552  1995.377958  3838.244082   
max        2155.672404  3839.283205    0.997360  2159.782144  3843.716391   

scorer                 
bodyparts              
coords     likelihood  
count      396.000000  
mean         0.122058  
std          0.173181  
min          0.000364  
25%          0.009609  
50%          0.044030  
75%          0.171100  
max          0.900990

In [76]:
df = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\front4k60fpszoom3.0_Trim-nv-2019-06-03\videos\front1080p240fpszoom1.0DeepCut_resnet50_front4k60fpszoom3.0_TrimJun3shuffle1_286000.h5")
df.describe()

scorer    DeepCut_resnet50_front4k60fpszoom3.0_TrimJun3shuffle1_286000  \
bodyparts                                                     LeftHand   
coords                                                               x   
count                                           15512.000000             
mean                                              476.662807             
std                                               199.051516             
min                                                -3.072265             
25%                                               556.934558             
50%                                               571.585048             
75%                                               572.115142             
max                                               576.400276             

scorer                                                             \
bodyparts                                 RightHand                 
coords                y    likelihood             x             y   
count      15512.000000  15512.000000  15512.000000  15512.000000   
mean          56.168319      0.000372     94.257165    269.637234   
std          113.435357      0.000808    199.582534    103.977182   
min           -0.002832      0.000023     -1.198687     -0.875000   
25%            1.973024      0.000192      4.560107    284.634581   
50%            2.359020      0.000286      5.130947    317.105902   
75%            4.928002      0.000423      5.801471    317.801870   
max          322.130518      0.046496    573.605843    321.000363   

scorer                                                             \
bodyparts                        Nose                               
coords       likelihood             x             y    likelihood   
count      15512.000000  15512.000000  15512.000000  15512.000000   
mean           0.001873    503.875873     39.304025      0.000522   
std            0.002485    172.544028     96.968325      0.004167   
min            0.000019     -1.059059     -0.550313      0.000017   
25%            0.000632    571.378365      0.997769      0.000229   
50%            0.001176    571.993842      1.213735      0.000329   
75%            0.002189    572.493355      1.552477      0.000470   
max            0.067399    575.610920    321.239240      0.320831   

scorer                                               
bodyparts        Pellet                              
coords                x             y    likelihood  
count      15512.000000  15512.000000  15512.000000  
mean         459.721134     70.262810      0.001173  
std          220.905013    127.125900      0.006426  
min           -0.952420     -1.799350      0.000008  
25%          564.175327      1.154488      0.000486  
50%          571.777898      1.417860      0.000696  
75%          572.075551      1.793241      0.000972  
max          574.988145    321.820386      0.218525